In [1]:
import requests
import bs4
import pandas as pd
import re

In [13]:
informazioni_geografiche = pd.read_csv("informazioni_geografiche.csv")
informazioni_geografiche.head()

,sigla_provincia,Comune,Provincia,Regione,Provincie nella regione (N°),Comuni nella regione (N°),Superficie (kmq) della Regione,Macro Zona della Regione,Zona Precisa della Regione
0,TO,Agliè,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
1,TO,Airasca,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
2,TO,Ala di Stura,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
3,TO,Albiano d'Ivrea,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest
4,TO,Almese,Torino,Piemonte,8,1180,"25391,6652",Nord,Nord-Ovest


In [14]:
georef = pd.read_csv('georef-italy-comune.csv', sep=None, engine='python')
georef = georef[['Official Name Comune', 'Official Name Provincia/Città metropolitana', 'Official Name Regione', 'Official Name Ripartizione geografica']]
georef = georef.rename(columns={'Official Name Comune': 'Comune', 'Official Name Provincia/Città metropolitana': 'Provincia',
                                'Official Name Regione': 'Regione', 'Official Name Ripartizione geografica': 'Ripartizione geografica'})
georef.head()

,Comune,Provincia,Regione,Ripartizione geografica
0,Bisacquino,Palermo,Sicilia,Isole
1,Elmas,Cagliari,Sardegna,Isole
2,Benetutti,Sassari,Sardegna,Isole
3,Talana,Nuoro,Sardegna,Isole
4,Campo Calabro,Reggio di Calabria,Calabria,Sud


In [15]:
georef.dropna(inplace=True)

In [16]:
def imposta_zona(valore):
    zona = informazioni_geografiche.loc[informazioni_geografiche["Regione"] == valore, "Macro Zona della Regione"]
    
    # Se la Serie è vuota, restituisci un valore di default (es. "Sconosciuto" o None)
    return str(zona.iloc[0]) if not zona.empty else f"Sconosciuto"

georef['Macro Zona della Regione'] = georef['Regione'].apply(imposta_zona)

def imposta_sigla(valore):
    sigla = informazioni_geografiche.loc[informazioni_geografiche["Provincia"] == valore, "sigla_provincia"]
    
    # Se la Serie è vuota, restituisci un valore di default (es. "Sconosciuto" o None)
    return str(sigla.iloc[0]) if not sigla.empty else f"Sconosciuto"

georef['sigla_provincia'] = georef['Provincia'].apply(imposta_sigla)

In [17]:
georef[georef['sigla_provincia'] == 'Sconosciuto']

,Comune,Provincia,Regione,Ripartizione geografica,Macro Zona della Regione,sigla_provincia
4,Campo Calabro,Reggio di Calabria,Calabria,Sud,Sud,Sconosciuto
22,Brusciano,Napoli,Campania,Sud,Sud,Sconosciuto
43,Predappio,Forli'-Cesena,Emilia-Romagna,Nord-Est,Nord,Sconosciuto
96,Rocca San Casciano,Forli'-Cesena,Emilia-Romagna,Nord-Est,Nord,Sconosciuto
112,Portici,Napoli,Campania,Sud,Sud,Sconosciuto
...,...,...,...,...,...,...
7862,San Polo d'Enza,Reggio nell'Emilia,Emilia-Romagna,Nord-Est,Nord,Sconosciuto
7898,Saint-Nicolas,Aosta,Valle d'Aosta,Nord-Ovest,Sconosciuto,Sconosciuto
7899,Saint-Denis,Aosta,Valle d'Aosta,Nord-Ovest,Sconosciuto,Sconosciuto
7901,Meltina,Bolzano,Trentino-Alto Adige,Nord-Est,Nord,Sconosciuto


In [18]:
georef#['Provincia'].nunique()

,Comune,Provincia,Regione,Ripartizione geografica,Macro Zona della Regione,sigla_provincia
0,Bisacquino,Palermo,Sicilia,Isole,Sud,PA
1,Elmas,Cagliari,Sardegna,Isole,Ovest,CA
2,Benetutti,Sassari,Sardegna,Isole,Ovest,SS
3,Talana,Nuoro,Sardegna,Isole,Ovest,NU
4,Campo Calabro,Reggio di Calabria,Calabria,Sud,Sud,Sconosciuto
...,...,...,...,...,...,...
7899,Saint-Denis,Aosta,Valle d'Aosta,Nord-Ovest,Sconosciuto,Sconosciuto
7900,Mignano Monte Lungo,Caserta,Campania,Sud,Sud,CE
7901,Meltina,Bolzano,Trentino-Alto Adige,Nord-Est,Nord,Sconosciuto
7902,Gargazzone,Bolzano,Trentino-Alto Adige,Nord-Est,Nord,Sconosciuto


In [19]:

#georef[georef['sigla_provincia'].isna()]

In [ ]:
#georef.to_csv("informazioni_geografiche.csv", index=False)

In [21]:
georef[georef['Comune'] == 'Solarolo']

,Comune,Provincia,Regione,Ripartizione geografica,Macro Zona della Regione,sigla_provincia
1871,Solarolo,Ravenna,Emilia-Romagna,Nord-Est,Nord,RA


---